# Program do szeregów Fouriera


In [44]:
from sympy import symbols, init_printing, pi
from sympy.calculus.util import continuous_domain
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

x, nn = symbols("x, nn", real=True)
init_printing(use_unicode=True)

### Funkcja do szeregów Fouriera nr 1 - F1

---


In [45]:
def fourier1(f, n, draw=False, n_to_draw=[]):
    # W1: wyliczenie składowych
    an_formula = 1 / pi * sp.integrate(f * sp.cos(nn * x), (x, -pi, pi))
    bn_formula = 1 / pi * sp.integrate(f * sp.sin(nn * x), (x, -pi, pi))

    # wyliczenia an
    an_list = [an_formula.subs(nn, ni) for ni in range(0, n + 1)]
    # wyliczenia bn
    bn_list = [bn_formula.subs(nn, ni) for ni in range(1, n + 1)]

    skladowe = []
    for ni in range(1, n + 1):
        skladowe.append(
            an_list[ni]  # an
            * sp.cos(ni * x)
            + bn_list[ni - 1]  # bn
            * sp.sin(ni * x)
        )

    # W2: wyliczenie n-tego przybliżenia
    n_aprox = (
        an_list[0] / 2  # a0
        + sum(skladowe)
    ).simplify()

    if not draw:
        return skladowe, n_aprox, an_list, bn_list

    domain = continuous_domain(f, x, sp.S.Reals)
    x_vals = [x for x in np.linspace(float(-pi), float(pi), 200) if x in domain]
    f_vals = [f.subs(x, val) for val in x_vals]
    fmax, fmin = int(max(f_vals)) + 2, int(min(f_vals)) - 2
    n_aprox_vals = [n_aprox.subs(x, val) for val in x_vals]

    # W3: generowanie wykresu n-tego przybliżenia i f
    plt.plot(x_vals, f_vals, label="f", color="blue")
    plt.plot(x_vals, n_aprox_vals, label="N-aprox", color="red", linestyle="dashed")
    plt.ylim(fmin, fmax)
    plt.title("Funkcja f i n-tego przybliżenia")
    plt.legend()

    plt.show()

    # generowanie wykresów skladowych
    plt.plot(x_vals, n_aprox_vals, label="N-aprox", color="red", linewidth=2)
    for i, func in enumerate(skladowe):
        if func == 0 or (len(n_to_draw) != 0 and i + 1 not in n_to_draw):
            continue
        func_vals = [func.subs(x, val) for val in x_vals]
        plt.plot(x_vals, func_vals, label=f"N{i+1}", color=f"C{i}", linestyle="dashed")

    plt.ylim(fmin, fmax)
    plt.title("Wykresy składowych i n-tego przybliżenia")
    plt.legend()

    plt.show()

    return skladowe, n_aprox, an_list, bn_list

### Funkcja do szeregów Fouriera nr 2 - F2

---


In [46]:
def fourier2(f, n1, n2, draw=False, n_to_draw=[]):
    # W1: wyliczenie składowych
    skladowe = []
    for ni in range(1, n2 + 1):
        skladowe.append(
            (1 / pi * sp.integrate(f * sp.cos(ni * x), (x, -pi, pi)))  # an
            * sp.cos(ni * x)
            + (1 / pi * sp.integrate(f * sp.sin(ni * x), (x, -pi, pi)))  # bn
            * sp.sin(ni * x)
        )

    przyblizenia = []
    for ni in range(n1, n2 + 1):
        przyblizenia.append(
            (1 / pi * sp.integrate(f * sp.cos(0 * x), (x, -pi, pi))) / 2  # a0
            + sum(skladowe[:ni])
        )

    if not draw:
        return przyblizenia

    # W2: narysowanie wykresów n-tych przybliżeń
    domain = continuous_domain(f, x, sp.S.Reals)
    x_vals = [x for x in np.linspace(-4, 4, 200) if x in domain]
    f_vals = [f.subs(x, val) for val in x_vals]
    fmax, fmin = int(max(f_vals)) + 2, int(min(f_vals)) - 2

    plt.plot(x_vals, f_vals, label="f", color="blue")
    for i, func in enumerate(przyblizenia):
        if func == 0 or (len(n_to_draw) != 0 and i + 1 not in n_to_draw):
            continue
        func_vals = [func.subs(x, val) for val in x_vals]
        plt.plot(x_vals, func_vals, label=f"N{i+n1}", color=f"C{i}", linestyle="dashed")

    plt.ylim(fmin, fmax)
    plt.title("Wykresy funkcji f i n-tych przybliżeń")
    plt.legend()

    plt.show()

    return przyblizenia

### Zadanie F3


In [47]:
f = sp.Piecewise(
    (x + pi, sp.And(-pi <= x, x < 0)),
    (x**2 - pi**2, sp.And(0 < x, x <= pi)),
    (0, True),
)
display(f)

⎧ x + π   for x ≥ -π ∧ x < 0
⎪                           
⎪ 2    2                    
⎨x  - π   for x ≤ π ∧ x > 0 
⎪                           
⎪   0         otherwise     
⎩                           

In [48]:
skladowe, n_approx, an_list, bn_list = fourier1(f, 9)

In [49]:
# wartości a
table_data = [(f"a{i}", an_list[i].evalf(5)) for i in range(len(an_list))]
pd.DataFrame(table_data, columns=["a", "Wartość"])

,a,Wartość
0,a0,-5.0089
1,a1,-1.3634
2,a2,0.50000
3,a3,-0.15149
4,a4,0.12500
5,a5,-0.054535
6,a6,0.055556
7,a7,-0.027824
8,a8,0.031250
9,a9,-0.016832


In [50]:
# wartości b
table_data = [(f"b{i+1}", bn_list[i].evalf(5)) for i in range(len(bn_list))]
pd.DataFrame(table_data, columns=["b", "Wartość"])

,b,Wartość
0,b1,-5.4148
1,b2,-2.0708
2,b3,-1.4277
3,b4,-1.0354
4,b5,-0.83850
5,b6,-0.69027
6,b7,-0.59537
7,b8,-0.51770
8,b9,-0.46192


In [51]:
przyblizenia = fourier2(f, 1, 8)

In [52]:
wybrane_wartosci = [-3 * pi / 4, -pi / 2, -pi / 4, pi / 4, pi / 2, 3 * pi / 4]
wybrane_przyblizenia = [2, 5, 8]

data = {"Fun": [f"f{i}" for i in wybrane_przyblizenia] + ["f(x)"]}
for val in wybrane_wartosci:
    data[f"{val}"] = [
        przyblizenia[ii - 1].subs(x, val).evalf(5) for ii in wybrane_przyblizenia
    ] + [f.subs(x, val).evalf(5)]

pd.DataFrame.from_dict(data)

,Fun,-3*pi/4,-pi/2,-pi/4,pi/4,pi/2,3*pi/4
0,f2,0.21765,2.4104,2.4311,-9.3682,-8.4193,-3.2985
1,f5,0.36359,1.9462,2.8684,-9.7641,-7.7051,-3.9858
2,f8,0.68379,1.3265,1.7688,-8.6413,-7.1341,-4.2041
3,f(x),0.78540,1.5708,2.3562,-9.2527,-7.4022,-4.3180


### Zadanie F4


In [53]:
f = sp.Piecewise(
    (x + pi, sp.And(-pi <= x, x < 0)),
    (x - pi, sp.And(0 < x, x <= pi)),
    (0, True),
)
display(f)

⎧x + π  for x ≥ -π ∧ x < 0
⎪                         
⎨x - π  for x ≤ π ∧ x > 0 
⎪                         
⎩  0        otherwise     

In [54]:
skladowe, n_approx, an_list, bn_list = fourier1(f, 6)

In [55]:
# wartości a
table_data = [(f"a{i}", an_list[i].evalf(5)) for i in range(len(an_list))]
pd.DataFrame(table_data, columns=["a", "Wartość"])

,a,Wartość
0,a0,0
1,a1,0
2,a2,0
3,a3,0
4,a4,0
5,a5,0
6,a6,0


In [56]:
# wartości b
table_data = [(f"b{i+1}", bn_list[i].evalf(5)) for i in range(len(bn_list))]
pd.DataFrame(table_data, columns=["b", "Wartość"])

,b,Wartość
0,b1,-2.0000
1,b2,-1.0000
2,b3,-0.66667
3,b4,-0.50000
4,b5,-0.40000
5,b6,-0.33333


In [57]:
przyblizenia = fourier2(f, 1, 6)

In [58]:
wybrane_wartosci = [-3 * pi / 4, -pi / 2, -pi / 4, pi / 4, pi / 2, 3 * pi / 4]
wybrane_przyblizenia = [2, 4, 6]

data = {"Fun": [f"f{i}" for i in wybrane_przyblizenia] + ["f(x)"]}
for val in wybrane_wartosci:
    data[f"{val}"] = [
        przyblizenia[ii - 1].subs(x, val).evalf(5) for ii in wybrane_przyblizenia
    ] + [f.subs(x, val).evalf(5)]

pd.DataFrame.from_dict(data)

,Fun,-3*pi/4,-pi/2,-pi/4,pi/4,pi/2,3*pi/4
0,f2,0.41421,2.0000,2.4142,-2.4142,-2.0000,-0.41421
1,f4,0.88562,1.3333,2.8856,-2.8856,-1.3333,-0.88562
2,f6,0.93611,1.7333,2.2694,-2.2694,-1.7333,-0.93611
3,f(x),0.78540,1.5708,2.3562,-2.3562,-1.5708,-0.78540


### Zadanie F5


In [59]:
f = sp.Piecewise(
    (-x - pi, sp.And(-pi <= x, x <= 0)),
    (x - pi, sp.And(0 < x, x <= pi)),
)
display(f)

⎧-x - π  for x ≥ -π ∧ x ≤ 0
⎨                          
⎩x - π   for x ≤ π ∧ x > 0 

In [60]:
skladowe, n_approx, an_list, bn_list = fourier1(f, 6)

In [61]:
# wartości a
table_data = [(f"a{i}", an_list[i].evalf(5)) for i in range(len(an_list))]
pd.DataFrame(table_data, columns=["a", "Wartość"])

,a,Wartość
0,a0,-3.1416
1,a1,-1.2732
2,a2,0
3,a3,-0.14147
4,a4,0
5,a5,-0.050930
6,a6,0


In [62]:
# wartości b
table_data = [(f"b{i+1}", bn_list[i].evalf(5)) for i in range(len(bn_list))]
pd.DataFrame(table_data, columns=["b", "Wartość"])

,b,Wartość
0,b1,0
1,b2,0
2,b3,0
3,b4,0
4,b5,0
5,b6,0


In [63]:
przyblizenia = fourier2(f, 1, 6)

In [64]:
wybrane_wartosci = [-3 * pi / 4, -pi / 2, -pi / 4, pi / 4, pi / 2, 3 * pi / 4]
wybrane_przyblizenia = [1, 3, 5]

data = {"Fun": [f"f{i}" for i in wybrane_przyblizenia] + ["f(x)"]}
for val in wybrane_wartosci:
    data[f"{val}"] = [
        przyblizenia[ii - 1].subs(x, val).evalf(5) for ii in wybrane_przyblizenia
    ] + [f.subs(x, val).evalf(5)]

pd.DataFrame.from_dict(data)

,Fun,-3*pi/4,-pi/2,-pi/4,pi/4,pi/2,3*pi/4
0,f1,-0.67048,-1.5708,-2.4711,-2.4711,-1.5708,-0.67048
1,f3,-0.77052,-1.5708,-2.3711,-2.3711,-1.5708,-0.77052
2,f5,-0.80653,-1.5708,-2.3351,-2.3351,-1.5708,-0.80653
3,f(x),-0.78540,-1.5708,-2.3562,-2.3562,-1.5708,-0.78540
